## Document Understanding
documentation:  https://docs.oracle.com/en-us/iaas/Content/document-understanding/using/home.htm

slack: #oci_ai_document_service_users or #igiu-innovation-lab if you have quesytion on service

issues with code: #igiu-ai-learning

## Import Libraries

In [1]:
import oci
import io
from oci.object_storage import ObjectStorageClient
import json,os,io

## Set input variables

In [2]:
#####
#make sure your sandbox.json file is setup for your environment. You might have to specify the full path depending on  your `cwd` 
# you can also try making your cwd ofr jupyter match your workspace python code: 
# vscopde menu -> Settings > Extensions > Jupyter > Notebook File Root
# change from ${fileDirname} to ${workspaceFolder}
#####

#SANDBOX_CONFIG_FILE = "~/work/code/python/workshop/sandbox.json"
SANDBOX_CONFIG_FILE = "sandbox.json"

FILE_TO_ANALYZE = "./vision/dussera-b.jpg"

# Read the config file

In [3]:

scfg = None
# read the sandbox config 
with open(os.path.expanduser(SANDBOX_CONFIG_FILE), 'r') as f:
                scfg=  json.load(f)

oci_cfg = oci.config.from_file(os.path.expanduser(scfg["oci"]["configFile"]),scfg["oci"]["profile"])
bucket_cfg = scfg["bucket"]
namespace = bucket_cfg["namespace"]
bucketName =  bucket_cfg["bucketName"]
filename = os.path.basename(FILE_TO_ANALYZE)
prefix = bucket_cfg['prefix']
compartmentId =scfg["oci"]["compartment"] 
    


## Upload file  

This is an optional step. If the file is already uploaded, no need to upload it again.

In [ ]:
object_storage_client = ObjectStorageClient(oci_cfg)
print(f"Uploading file {FILE_TO_ANALYZE} ...")
object_storage_client.put_object(bucket_cfg["namespace"], 
                                bucket_cfg["bucketName"], 
                                f"{bucket_cfg['prefix']}/{os.path.basename(FILE_TO_ANALYZE)}", 
                                io.open(FILE_TO_ANALYZE,'rb'))
print("Upload completed !")

## Create a callback function to print the status

In [5]:
def create_processor_job_callback(times_called, response):
    print("Waiting for processor lifecycle state to go into succeeded state:", response.data)
    

## Create AI service vision client

In [6]:
dus_client = oci.ai_document.AIServiceDocumentClientCompositeOperations(oci.ai_document.AIServiceDocumentClient(config=oci_cfg))

## Setup  features to be analyzed 
You can specify the features you want to call. In the below code we are calling all the features

In [7]:
features = [ oci.ai_document.models.DocumentClassificationFeature(),
            oci.ai_document.models.DocumentLanguageClassificationFeature(), 
            oci.ai_document.models.DocumentKeyValueExtractionFeature(),
            oci.ai_document.models.DocumentTableExtractionFeature(),
            oci.ai_document.models.DocumentTextExtractionFeature()
            ]

## Setup input location

In [8]:
input_location = oci.ai_document.models.ObjectLocation()
input_location.namespace_name = namespace
input_location.bucket_name =   bucketName
input_location.object_name = f"{prefix}/{filename}"

## Setup output location

In [9]:
output_location = oci.ai_document.models.OutputLocation()
output_location.namespace_name =  namespace
output_location.bucket_name =  bucketName
output_location.prefix = prefix

## Create image job 

In [13]:
display_name = f"{prefix}-test"
job_details = oci.ai_document.models.CreateProcessorJobDetails(
                                    display_name=display_name,
                                    compartment_id=compartmentId,
                                    input_location=oci.ai_document.models.ObjectStorageLocations(object_locations=[input_location]),
                                    output_location= output_location,
                                    processor_config=oci.ai_document.models.GeneralProcessorConfig(features=features)
                                    )
    

## Job in progress
The job progress is tracked till completion with an interval of 5 seconds.

In [ ]:
processor= dus_client.create_processor_job_and_wait_for_state(
    create_processor_job_details=job_details,
    wait_for_states=[oci.ai_document.models.ProcessorJob.LIFECYCLE_STATE_SUCCEEDED],
    waiter_kwargs={"wait_callback": create_processor_job_callback})

## Get response json from object storage
The output can be found in the output location specified or it can be saved in ```output.json``` file by running the below cell

In [ ]:
print(f"processor call succeeded with status: {[processor.status]} and request_id: {processor.request_id}.")
processor_job: oci.ai_document.models.ProcessorJob = processor.data

print(f"Getting result json from the output_location {processor_job.id}")

object_storage_client = oci.object_storage.ObjectStorageClient(config=oci_cfg)
get_object_response = object_storage_client.get_object(namespace_name=namespace,
                                                       bucket_name=bucketName,
                                                       object_name=f"{prefix}/{processor_job.id}/{namespace}_{bucketName}/results/{prefix}/{filename}.json")

print(str(get_object_response.data.content.decode()))


## Exercise 1 : Dcument Validator

1. Create a document in PowerPoint with
    * Name
    * Address
    * Dates 
    * Create date
    * Expiry date
    * Signature

1. Save it as an image 

1. Use Document Understanding & LLM service to validate
    *  is on correct name
    *  is on correct address
    *  is not expired
    *  has a signature



## Exercise 2 : Form Filler

1. Upload a receipt 
1. Fill out a expense report based on the receipt 
    *  Image an expense resporr as an multi line f string for simplicity
